In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import numpy as np
import struct

In [13]:

train_images_path = './Data/fashion mnist/train-images-idx3-ubyte.gz'
train_labels_path = './Data/fashion mnist/train-labels-idx1-ubyte.gz'
test_images_path = './Data/fashion mnist/t10k-images-idx3-ubyte.gz'
test_labels_path = './Data/fashion mnist/t10k-labels-idx1-ubyte.gz'

def read_idx_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(filepath):
    with gzip.open(filepath, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = read_idx_images(train_images_path)
train_labels = read_idx_labels(train_labels_path)
test_images = read_idx_images(test_images_path)
test_labels = read_idx_labels(test_labels_path)

print(f'Train images shape: {train_images.shape}')  # Should be (60000, 28, 28)
print(f'Train labels shape: {train_labels.shape}')  # Should be (60000,)
print(f'Test images shape: {test_images.shape}')    # Should be (10000, 28, 28)
print(f'Test labels shape: {test_labels.shape}')    # Should be (10000,)


NameError: name 'gzip' is not defined

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 100)   # First hidden layer with 100 nodes
        self.relu = nn.ReLU()              # ReLU activation function
        self.dropout = nn.Dropout(0.3)     # Dropout with 30% probability
        self.fc2 = nn.Linear(100, 10)      # Output layer for 10 classes
    
    def forward(self, x):
        x = x.view(-1, 28*28)              # Flatten the image
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.0001)  # L2 regularization with weight_decay

# 4. Training the Model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


In [ ]:
model.eval()  # Set model to evaluation mode
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 6. Generating and Plotting Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues', xticklabels=train_set.classes, yticklabels=train_set.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Fashion-MNIST Classification')
plt.show()

# 7. Analyzing Most Confused Classes
conf_matrix_sum = conf_matrix.sum(axis=1)
most_confused_classes = conf_matrix_sum.argsort()[-2:][::-1]
print(f'Two classes most confused with each other: {train_set.classes[most_confused_classes[0]]} and {train_set.classes[most_confused_classes[1]]}')